In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
from tqdm import tqdm
from itertools import permutations
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

c:\Anaconda3\envs\daycon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


사용 디바이스: cuda


In [2]:
def create_label_mappings():
    """24가지 순열에 대한 라벨 매핑 생성"""
    # 가능한 모든 순열 생성 (4! = 24개)
    all_permutations = list(permutations([0, 1, 2, 3]))
    
    # 순열 → 라벨 번호 매핑
    perm_to_label = {perm: idx for idx, perm in enumerate(all_permutations)}
    
    # 라벨 번호 → 순열 매핑 (예측시 사용)
    label_to_perm = {idx: perm for idx, perm in enumerate(all_permutations)}
    
    print(f"총 {len(all_permutations)}개의 순열 클래스 생성")
    print("예시 매핑:")
    for i in range(5):
        print(f"  라벨 {i}: {all_permutations[i]}")
    
    return perm_to_label, label_to_perm

# 순열 매핑 생성
perm_to_label, label_to_perm = create_label_mappings()

총 24개의 순열 클래스 생성
예시 매핑:
  라벨 0: (0, 1, 2, 3)
  라벨 1: (0, 1, 3, 2)
  라벨 2: (0, 2, 1, 3)
  라벨 3: (0, 2, 3, 1)
  라벨 4: (0, 3, 1, 2)


In [3]:
def prepare_roberta_data(train_df, perm_to_label):
    """RoBERTa용 데이터 준비"""
    processed_data = []
    
    for _, row in train_df.iterrows():
        # 4개 문장 추출
        sentences = [row[f"sentence_{i}"] for i in range(4)]
        
        # 정답 순열
        answer_tuple = tuple([row[f"answer_{i}"] for i in range(4)])
        
        # 문장들을 [SEP]로 연결
        text = " [SEP] ".join(sentences)
        
        # 순열을 라벨로 변환
        label = perm_to_label[answer_tuple]
        
        processed_data.append({
            "text": text,
            "label": label,
            "original_sentences": sentences,
            "answer": answer_tuple
        })
    
    return processed_data

def make_t5_input(row):
    """T5용 입력 데이터 생성"""
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    input_text = "문장을 순서대로 정렬하세요: " + " </s> ".join(sentences)
    answer = [row[f"answer_{i}"] for i in range(4)]
    target_text = " ".join(map(str, answer))  # 예: "0 3 1 2"
    return {"input": input_text, "target": target_text}

def augment_roberta_data(train_df, perm_to_label, multiplier=3):
    """데이터 증강으로 학습 데이터 늘리기"""
    augmented_data = []
    
    # 원본 데이터 처리
    original_data = prepare_roberta_data(train_df, perm_to_label)
    augmented_data.extend(original_data)
    
    # 증강 데이터 생성
    for _ in range(multiplier - 1):
        for _, row in train_df.iterrows():
            sentences = [row[f"sentence_{i}"] for i in range(4)]
            original_answer = [row[f"answer_{i}"] for i in range(4)]
            
            # 랜덤하게 문장 순서 섞기
            indices = list(range(4))
            np.random.shuffle(indices)
            
            # 새로운 순서의 문장들
            shuffled_sentences = [sentences[i] for i in indices]
            
            # 새로운 정답 계산
            new_answer = tuple([indices.index(original_answer[i]) for i in range(4)])
            
            # 텍스트 생성
            text = " [SEP] ".join(shuffled_sentences)
            label = perm_to_label[new_answer]
            
            augmented_data.append({
                "text": text,
                "label": label,
                "original_sentences": shuffled_sentences,
                "answer": new_answer
            })
    
    print(f"데이터 증강 완료: {len(original_data)} → {len(augmented_data)}")
    return augmented_data

print("데이터 전처리 함수 정의 완료!")

데이터 전처리 함수 정의 완료!


In [4]:
# 데이터 로드
print("데이터 로드 중...")
train_df = pd.read_csv('./train.csv')
print(f"원본 학습 데이터: {len(train_df)}개")

# 데이터 증강
print("\n데이터 증강 중...")
augmented_data = augment_roberta_data(train_df, perm_to_label, multiplier=3)

# 학습/검증 분할
print("\n학습/검증 데이터 분할...")
train_data, valid_data = train_test_split(
    augmented_data, 
    test_size=0.2, 
    random_state=42,
    stratify=[item["label"] for item in augmented_data]
)

print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(valid_data)}개")

# T5용 데이터 준비 (증강 데이터 사용)
print("\nT5용 데이터 준비 (증강 데이터 사용)...")

def convert_to_t5_format(augmented_data):
    """증강된 데이터를 T5 형식으로 변환"""
    t5_data = []
    for item in augmented_data:
        sentences = item['original_sentences']
        answer = item['answer']
        
        input_text = "문장을 순서대로 정렬하세요: " + " </s> ".join(sentences)
        target_text = " ".join(map(str, answer))
        
        t5_data.append({
            "input": input_text,
            "target": target_text
        })
    return t5_data

# 증강된 train_data, valid_data를 T5 형식으로 변환
t5_train_data = convert_to_t5_format(train_data)
t5_valid_data = convert_to_t5_format(valid_data)

t5_train_dataset = Dataset.from_pandas(pd.DataFrame(t5_train_data))
t5_valid_dataset = Dataset.from_pandas(pd.DataFrame(t5_valid_data))

print(f"T5 학습 데이터: {len(t5_train_data)}개 (증강된 데이터)")
print(f"T5 검증 데이터: {len(t5_valid_data)}개 (증강된 데이터)")

데이터 로드 중...
원본 학습 데이터: 7351개

데이터 증강 중...
데이터 증강 완료: 7351 → 22053

학습/검증 데이터 분할...
학습 데이터: 17642개
검증 데이터: 4411개

T5용 데이터 준비 (증강 데이터 사용)...
T5 학습 데이터: 17642개 (증강된 데이터)
T5 검증 데이터: 4411개 (증강된 데이터)


In [6]:
print("T5 모델 및 토크나이저 로딩...")

# T5 토크나이저 및 모델 로딩
t5_model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name, cache_dir='C:/huggingface_cache')
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name, cache_dir='C:/huggingface_cache')
t5_model.to(device)

print("✅ T5 모델 로드 완료!")

# T5 토크나이징 함수
def tokenize_t5(example):
    model_inputs = t5_tokenizer(example["input"], max_length=512, truncation=True, padding="max_length")
    labels = t5_tokenizer(example["target"], max_length=16, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# T5 데이터셋 토크나이징
print("T5 데이터셋 토크나이징 중...")
t5_tokenized_train = t5_train_dataset.map(tokenize_t5, batched=True)
t5_tokenized_valid = t5_valid_dataset.map(tokenize_t5, batched=True)
print("T5 토크나이징 완료!")

T5 모델 및 토크나이저 로딩...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


✅ T5 모델 로드 완료!
T5 데이터셋 토크나이징 중...


Map: 100%|██████████| 4411/4411 [00:02<00:00, 1715.26 examples/s]

T5 토크나이징 완료!


In [14]:
print("T5 모델 학습 시작...")

# EarlyStoppingCallback 추가
from transformers import EarlyStoppingCallback

# T5 학습 설정 (RTX 2070 Super 8GB 최적화)
t5_training_args = TrainingArguments(
    output_dir="./t5_results",
    learning_rate=3e-5,  # 조금 높여서 빠른 수렴
    
    # 메모리 최적화 배치 설정
    per_device_train_batch_size=4,  # 16 → 4로 대폭 축소
    per_device_eval_batch_size=8,   # 32 → 8로 축소
    gradient_accumulation_steps=4,  # 실제 배치 크기: 4*4=16
    
    num_train_epochs=15,  # 20 → 15로 단축
    
    # 메모리 최적화 설정
    fp16=True,  # 메모리 50% 절약
    gradient_checkpointing=True,  # 메모리 절약 (속도 20% 감소)
    dataloader_pin_memory=False,  # 메모리 절약
    dataloader_num_workers=2,     # 4 → 2로 축소
    
    # 평가 및 저장 (조기 종료를 위해 더 자주)
    eval_strategy="steps",
    eval_steps=200,  # 100 → 200으로 늘려서 메모리 압박 완화
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,  # 3 → 2로 축소 (디스크 공간 절약)
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # 로깅
    logging_steps=100,
    report_to=None,
    
    # 추가 메모리 최적화
    remove_unused_columns=True,
    dataloader_drop_last=True,
)

# T5 Trainer 정의 (EarlyStoppingCallback 추가)
t5_trainer = Trainer(
    model=t5_model,
    args=t5_training_args,
    train_dataset=t5_tokenized_train,
    eval_dataset=t5_tokenized_valid,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 3번 연속 개선 없으면 중단
)

print("T5 학습 시작...")
t5_trainer.train()

T5 모델 학습 시작...
T5 학습 시작...


Step,Training Loss,Validation Loss
200,0.200400,0.183419
400,0.198700,0.185852
600,0.195400,0.180419
800,0.194100,0.174857
1000,0.187600,0.171759
1200,0.180800,0.170843
1400,0.181800,0.170376
1600,0.179600,0.170007
1800,0.179800,0.169599
2000,0.175500,0.166112


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=7600, training_loss=0.16754526602594477, metrics={'train_runtime': 5206.008, 'train_samples_per_second': 50.832, 'train_steps_per_second': 3.178, 'total_flos': 1.6451066652524544e+16, 'train_loss': 0.16754526602594477, 'epoch': 6.890702947845805})

In [20]:
# T5 모델 저장
print("T5 모델 저장 중...")
t5_trainer.save_model("./t5_results/final_model")
t5_tokenizer.save_pretrained("./t5_results/final_model")
print("✅ T5 학습 및 저장 완료!")

T5 모델 저장 중...
✅ T5 학습 및 저장 완료!


In [11]:
print("RoBERTa 모델 및 토크나이저 로딩...")

# RoBERTa 모델 설정
roberta_model_name = "klue/roberta-base"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name, cache_dir='C:/huggingface_cache')
print(f"토크나이저 로드 성공: {type(roberta_tokenizer).__name__}")

roberta_model = AutoModelForSequenceClassification.from_pretrained(
    roberta_model_name,
    num_labels=24,  # 24가지 순열
    cache_dir='C:/huggingface_cache'
)
roberta_model.to(device)

print("✅ RoBERTa 모델 로드 완료!")

# RoBERTa 데이터셋 준비
train_df_processed = pd.DataFrame(train_data)
valid_df_processed = pd.DataFrame(valid_data)

train_dataset = Dataset.from_pandas(train_df_processed)
valid_dataset = Dataset.from_pandas(valid_df_processed)

# RoBERTa 토크나이징 함수
def tokenize_roberta(examples):
    return roberta_tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512
    )

# RoBERTa 데이터셋 토크나이징
print("RoBERTa 데이터셋 토크나이징 중...")
roberta_tokenized_train = train_dataset.map(tokenize_roberta, batched=True)
roberta_tokenized_valid = valid_dataset.map(tokenize_roberta, batched=True)

# 불필요한 컬럼 제거
roberta_tokenized_train = roberta_tokenized_train.remove_columns(["text", "original_sentences", "answer"])
roberta_tokenized_valid = roberta_tokenized_valid.remove_columns(["text", "original_sentences", "answer"])

print("RoBERTa 토크나이징 완료!")

RoBERTa 모델 및 토크나이저 로딩...
토크나이저 로드 성공: BertTokenizerFast


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ RoBERTa 모델 로드 완료!
RoBERTa 데이터셋 토크나이징 중...


Map: 100%|██████████| 4411/4411 [00:00<00:00, 8505.29 examples/s]

RoBERTa 토크나이징 완료!


In [16]:
print("RoBERTa 모델 학습 시작...")

# EarlyStoppingCallback import (이미 위에서 했지만 안전하게)
from transformers import EarlyStoppingCallback

# 평가 메트릭 정의
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# RoBERTa 학습 설정 (RTX 2070 Super 8GB 최적화)
roberta_training_args = TrainingArguments(
    output_dir="./roberta_results",
    learning_rate=3e-5,  # 조금 높여서 빠른 수렴
    
    # 메모리 최적화 배치 설정
    per_device_train_batch_size=16,  # 64 → 16로 축소
    per_device_eval_batch_size=32,   # 128 → 32로 축소  
    gradient_accumulation_steps=4,   # 2 → 4로 증가, 실제 배치: 16*4=64
    
    num_train_epochs=15,  # 20 → 15로 단축
    
    # 메모리 최적화 설정
    fp16=True,
    gradient_checkpointing=True,  # 메모리 절약
    dataloader_pin_memory=False,  # 메모리 절약
    dataloader_num_workers=2,     # 6 → 2로 축소
    
    # 평가 및 저장 (조기 종료를 위해 더 자주)
    eval_strategy="steps",
    eval_steps=200,  # 100 → 200으로 늘려서 메모리 압박 완화
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,  # 3 → 2로 축소
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # 정확도 기준
    greater_is_better=True,
    
    # 학습 최적화
    warmup_steps=200,  # 150 → 200
    weight_decay=0.01,
    
    # 로깅
    logging_dir='./logs',
    logging_steps=50,  # 20 → 50
    report_to=None,
    
    # 추가 메모리 최적화
    remove_unused_columns=True,
    dataloader_drop_last=True,
)

# 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

# RoBERTa Trainer 설정 (EarlyStoppingCallback 추가)
roberta_trainer = Trainer(
    model=roberta_model,
    args=roberta_training_args,
    train_dataset=roberta_tokenized_train,
    eval_dataset=roberta_tokenized_valid,
    tokenizer=roberta_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 3번 연속 개선 없으면 중단
)

print("RoBERTa 학습 시작...")
roberta_trainer.train()


RoBERTa 모델 학습 시작...
RoBERTa 학습 시작...


Step,Training Loss,Validation Loss,Accuracy
200,2.580900,2.467889,0.122719
400,1.822300,1.577760,0.438412
600,0.826100,0.842864,0.667427
800,0.588100,0.545402,0.818203
1000,0.388300,0.419764,0.868385
1200,0.221400,0.336767,0.907162
1400,0.175700,0.303742,0.919024
1600,0.152400,0.263528,0.929516
1800,0.103400,0.216563,0.943203
2000,0.067400,0.219737,0.950730


TrainOutput(global_step=4140, training_loss=0.405164846583553, metrics={'train_runtime': 2824.0126, 'train_samples_per_second': 93.707, 'train_steps_per_second': 1.466, 'total_flos': 1.9441645636299264e+16, 'train_loss': 0.405164846583553, 'epoch': 15.0})

In [21]:
# RoBERTa 모델 저장
print("RoBERTa 모델 저장 중...")
roberta_trainer.save_model("./roberta_results/final_model")
roberta_tokenizer.save_pretrained("./roberta_results/final_model")
print("✅ RoBERTa 학습 및 저장 완료!")

RoBERTa 모델 저장 중...
✅ RoBERTa 학습 및 저장 완료!


In [22]:
print("학습된 모델들 로드 중...")

# 학습된 T5 모델 로드
print("학습된 T5 모델 로딩...")
t5_tokenizer = T5Tokenizer.from_pretrained("./t5_results/final_model")
t5_model = T5ForConditionalGeneration.from_pretrained("./t5_results/final_model")
t5_model.to(device)
t5_model.eval()
print("✅ 학습된 T5 모델 로드 완료!")

# 학습된 RoBERTa 모델 로드
print("학습된 RoBERTa 모델 로딩...")
roberta_tokenizer = AutoTokenizer.from_pretrained("./roberta_results/final_model")
roberta_model = AutoModelForSequenceClassification.from_pretrained("./roberta_results/final_model")
roberta_model.to(device)
roberta_model.eval()
print("✅ 학습된 RoBERTa 모델 로드 완료!")

print("\n🎉 모든 모델 학습 및 로드 완료!")

# GPU 메모리 체크
def print_gpu_utilization():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        usage_percent = (allocated / total) * 100
        print(f"GPU 메모리: {allocated:.2f}GB 사용 / {total:.2f}GB 전체 ({usage_percent:.1f}% 사용)")
    else:
        print("CUDA 사용 불가")

print_gpu_utilization()

학습된 모델들 로드 중...
학습된 T5 모델 로딩...
✅ 학습된 T5 모델 로드 완료!
학습된 RoBERTa 모델 로딩...
✅ 학습된 RoBERTa 모델 로드 완료!

🎉 모든 모델 학습 및 로드 완료!
GPU 메모리: 3.05GB 사용 / 8.00GB 전체 (38.1% 사용)


In [23]:
def predict_t5(sentences, model, tokenizer, device):
    """T5 모델로 예측"""
    input_text = "문장을 순서대로 정렬하세요: " + " </s> ".join(sentences)
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding="longest",
        max_length=512
    ).to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=16,
            do_sample=True,      
            temperature=0.2,     
            top_p=0.9,            
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    
    try:
        order = list(map(int, decoded.strip().split()))
        if len(order) == 4 and set(order) == {0, 1, 2, 3}:
            return order, 1.0
        else:
            return [0, 1, 2, 3], 0.1
    except:
        return [0, 1, 2, 3], 0.1

def predict_roberta(sentences, model, tokenizer, label_to_perm, device):
    """RoBERTa 모델로 예측"""
    text = " [SEP] ".join(sentences)
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_label = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_label].item()
    
    predicted_order = list(label_to_perm[predicted_label])
    return predicted_order, confidence

def ensemble_predict_confidence_based(sentences, t5_model, t5_tokenizer, roberta_model, roberta_tokenizer, label_to_perm, device, confidence_threshold=0.7):
    """신뢰도 기반 앙상블 예측"""
    # 각 모델의 예측
    t5_pred, t5_conf = predict_t5(sentences, t5_model, t5_tokenizer, device)
    roberta_pred, roberta_conf = predict_roberta(sentences, roberta_model, roberta_tokenizer, label_to_perm, device)
    
    # RoBERTa 신뢰도가 임계값보다 높으면 RoBERTa 선택
    if roberta_conf >= confidence_threshold:
        return roberta_pred, roberta_conf, "roberta"
    else:
        return t5_pred, t5_conf, "t5"

print("앙상블 예측 함수 정의 완료!")

앙상블 예측 함수 정의 완료!


In [25]:
def evaluate_ensemble_performance(valid_data, confidence_threshold=0.7):
    """앙상블 성능 평가"""
    correct = 0
    total = len(valid_data)
    
    predictions = []
    confidences = []
    model_selections = []
    
    print(f"앙상블 성능 평가 중... (신뢰도 임계값: {confidence_threshold})")
    
    for example in tqdm(valid_data, desc="평가 진행"):
        sentences = example['original_sentences']
        true_order = list(example['answer'])
        
        pred_order, confidence, selected_model = ensemble_predict_confidence_based(
            sentences, t5_model, t5_tokenizer, roberta_model, roberta_tokenizer, 
            label_to_perm, device, confidence_threshold
        )
        
        predictions.append(pred_order)
        confidences.append(confidence)
        model_selections.append(selected_model)
        
        if pred_order == true_order:
            correct += 1
    
    accuracy = correct / total
    avg_confidence = np.mean(confidences)
    
    # 모델 선택 통계
    model_stats = Counter(model_selections)
    
    print(f"\n=== 앙상블 평가 결과 ===")
    print(f"정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"정답 개수: {correct}/{total}")
    print(f"평균 신뢰도: {avg_confidence:.4f}")
    print(f"모델 선택 통계: {dict(model_stats)}")
    
    return accuracy, predictions, confidences, model_selections

# 성능 평가 실행
accuracy, val_predictions, val_confidences, val_selections = evaluate_ensemble_performance(valid_data, confidence_threshold=0.7)

앙상블 성능 평가 중... (신뢰도 임계값: 0.7)


평가 진행: 100%|██████████| 4411/4411 [05:49<00:00, 12.63it/s]


=== 앙상블 평가 결과 ===
정확도: 0.9739 (97.39%)
정답 개수: 4296/4411
평균 신뢰도: 0.9967
모델 선택 통계: {'roberta': 4391, 't5': 20}


In [ ]:
print("테스트 데이터 예측 중...")

# 테스트 데이터 로드
test_df = pd.read_csv("./test.csv")
print(f"테스트 데이터: {len(test_df)}개")

predictions = []
confidences = []
model_selections = []

# 신뢰도 임계값 설정
optimal_threshold = 0.8
print(f"사용할 신뢰도 임계값: {optimal_threshold}")

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="앙상블 예측 진행"):
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    
    predicted_order, confidence, selected_model = ensemble_predict_confidence_based(
        sentences, t5_model, t5_tokenizer, roberta_model, roberta_tokenizer, 
        label_to_perm, device, optimal_threshold
    )
    
    predictions.append(predicted_order)
    confidences.append(confidence)
    model_selections.append(selected_model)

# 예측 통계
avg_confidence = np.mean(confidences)
model_stats = Counter(model_selections)

print(f"\n=== 테스트 예측 완료 ===")
print(f"평균 예측 신뢰도: {avg_confidence:.4f}")
print(f"모델 선택 통계: {dict(model_stats)}")

테스트 데이터 예측 중...
테스트 데이터: 1780개
사용할 신뢰도 임계값: 0.7


앙상블 예측 진행: 100%|██████████| 1780/1780 [02:23<00:00, 12.38it/s]


=== 테스트 예측 완료 ===
평균 예측 신뢰도: 0.9917
모델 선택 통계: {'roberta': 1755, 't5': 25}


In [26]:
print("Submission 파일 생성 중...")

# 샘플 제출 파일 로드
sample_submission = pd.read_csv("./sample_submission.csv")

# 예측 결과를 제출 형식으로 변환
for i in range(4):
    sample_submission[f"answer_{i}"] = [pred[i] for pred in predictions]

# 파일 저장
submission_filename = "ensemble_t5_roberta_finetuned.csv"
sample_submission.to_csv(submission_filename, index=False)

print(f"제출 파일 저장 완료: {submission_filename}")

# 최종 결과 요약
print(f"\n" + "="*60)
print(f"🎯 T5 + RoBERTa Fine-tuned 앙상블 완료!")
print(f"="*60)
print(f"📊 검증 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"🔮 평균 예측 신뢰도: {avg_confidence*100:.2f}%")
print(f"🎚️ 신뢰도 임계값: {optimal_threshold}")
print(f"🤖 테스트셋 모델 선택: {dict(model_stats)}")
print(f"📁 제출 파일: {submission_filename}")
print(f"="*60)

Submission 파일 생성 중...
제출 파일 저장 완료: ensemble_t5_roberta_finetuned.csv

🎯 T5 + RoBERTa Fine-tuned 앙상블 완료!
📊 검증 정확도: 0.9739 (97.39%)
🔮 평균 예측 신뢰도: 99.17%
🎚️ 신뢰도 임계값: 0.7
🤖 테스트셋 모델 선택: {'roberta': 1755, 't5': 25}
📁 제출 파일: ensemble_t5_roberta_finetuned.csv
